In [16]:
#SET UP#
#
#this script checks for missing and complete data for participants
#this is currently set up for the AOL study psytoolkit daily measures
#this checks for all 3 possible IDs (AOL study ID, task ID, and PHI ID) for data
# BP 8/9/20 
#
#TO USE: 
#
#1.
#download files from psytoolkit.org and put the zipped file in the data_dir directory (which can be set to whatever you want).  
#Name this zipped file 'data-{year}-{month}-{day}' to match the day it was downloaded. EXAMPLE: data-2020-09-01
#This script will unzip for you in the 'processing' cell
#The file above will be unzipped in the unzipped dir (can put wherever you want it to be -- but name it 'unzipped'). 
#Delete this unzipped dir when done with this script to save space. You will have the zipped file still. If you don't delete and run a new week, this script will crash from have too many files in the unzipped dir
#
#2.
#update the current_subs participants to whatever you need to check (can copy and paste subIDs from redcap output). Can set this sub_dir as anything you want.
#
#3.
#update the save_dir to save the file to whatever directory works best for you 
#
#4.
#download redcap file from redcap. Make sure it is 'raw data'
#put this file directly into redcap_dir (can be set to wherever you want)
#this script will select the most recent file in redcap_dir, so no need to delete old redcap files
#
#5.
#You only need to change directories or names in this first set up cell! Everything else should stay the same.
#run this script one at a time. Some cells may take a while to run so wait for 'done!' to print at the bottom of the cell each time you run a cell
#
#
#NOTES: 
#'missing' data may be due to a typo and may actually be present. 
#Someone should go into psytoolkit data and doublecheck that missing data is actually missing and not due to a typo when participant entered subID


#calling libraries
import pandas as pd
import numpy as np
import xlwt
from xlwt import Workbook
from os import listdir
from os.path import isfile, join
from zipfile import ZipFile 
from datetime import datetime  
from datetime import timedelta
from datetime import date

#date the data was downloaded (may not be the day this script is run)
data_date="2020-09-01"

#list of updated subs you want to check
#script will not run if you try to add a subject that does not exist on redcap
subs_filename = "current_subs.csv" 
subs_filename_recent_days = "current_subs_recent_days.csv" 

#number of days to check for recent daily tasks check -- last cell only. Can change this to whatever number of days you'd like to check
num_days_to_check=5

#directory where the current_subs file is stored. Must have double \\ not just a single \. Must have \\ at end of pathway
sub_dir= 'C:\\Users\\Becca\\Dropbox (UFL)\\Lab Studies\\AOL\\data back up checks\\data\\psytoolkit\\'
#directory where you want to save the check output files. Must have double \\ not just a single \. Must have \\ at end of pathway
save_dir = 'C:\\Users\\Becca\\Dropbox (UFL)\\Lab Studies\\AOL\\data back up checks\\data\\psytoolkit\\psytoolkit_output\\'
#directory where you will temporarily store unzipped files; not permanent and will detele unzips to save space
unzip_temp_dir = 'C:\\Users\\Becca\\Dropbox (UFL)\\Lab Studies\\AOL\\data back up checks\\data\\psytoolkit\\unzipped\\'
#subs_temp_dir = 'C:\\Users\\r.polk\\Dropbox (UFL)\\Lab Studies\\AOL\\data back up checks\\data\\psytoolkit\\temp_sub_files\\'
#directories where data is stored. Must have double \\ not just a single \. Must have \\ at end of pathway
redcap_dir = 'C:\\Users\\Becca\\Dropbox (UFL)\\Lab Studies\\AOL\\data back up checks\\data\\redcap\\'
psytoolkit_dir = 'C:\\Users\\Becca\\Dropbox (UFL)\\Lab Studies\\AOL\\data back up checks\\data\\psytoolkit\\'

#must make sure file names match and subID columns are correct
sub_id_column= "ID:1"
flanker_file_column = "flankertask:1"
questionnaire_file_column = "participant"
start_date_column = "TIME_start"

#listing all files in redcap_dir to select the most recent
all_red_dir_files = [f for f in listdir(redcap_dir) if isfile(join(redcap_dir, f))]
#sorting list of files
all_red_dir_files.sort(reverse=True)
#selecting top file (most recent date)
redcap_filename=all_red_dir_files[0]

#file names to save output. Update with date of most recent data (match the datafile)
save_name_present = 'psytoolkit_' + data_date + '.xls'
save_name_recent_days = 'psytoolkit_recent_days_' + data_date + '.xls'

print('done!')

done!


In [17]:
#PROCESSING#

#READING IN REDCAP FILE
#reading in the redcap file
col_list_redcap = ["record_id", "aol_phit_id", "aol_task_id", "screening_visit_remote_experimenter_checklist_complete", "v1_daily_task_start"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_filename,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_red_id = df_redcap["record_id"].apply(str).tolist()
df_phit_id = df_redcap["aol_phit_id"].astype(str).replace('\.0', '', regex=True).tolist() 
df_task_id = df_redcap["aol_task_id"].astype(str).replace('\.0', '', regex=True).tolist()
df_red_ses_check = df_redcap["screening_visit_remote_experimenter_checklist_complete"].tolist()
df_dailytasks_start = df_redcap["v1_daily_task_start"].apply(str).tolist()

#SELECTING PSYTOOLKIT FILE
#folder is the one based on date
psytoolkit_folder = 'data-' +  data_date

#UNZIPPING -- ONLY DO IF NEED TO UNZIP
#unzipping the file to unzip_temp_dir
#with ZipFile(psytoolkit_dir + psytoolkit_folder + '.zip', 'r') as zip_ref:
#    zip_ref.extractall(unzip_temp_dir)
    
    
#data to be read in from psytoolkit downloaded file    
psytoolkit_filename ='data.csv'

#READING IN PSYTOOLKIT FILE
#creating a list of 3 header names. Only these 3 columns will be read in
col_list_psytoolkit = [sub_id_column, flanker_file_column, start_date_column, questionnaire_file_column ]
#reading in the qualtrics file with only the header specified
df_psytoolkit = pd.read_csv(r'' + unzip_temp_dir + psytoolkit_filename,  usecols=col_list_psytoolkit)

#SORTING PSYTOOLKIT FILE
df_psytoolkit_sorted = df_psytoolkit.sort_values(by=[start_date_column,sub_id_column])

#opening a workbook to write results
wb1 = Workbook()

#creating empty temp dataframes to use later
df_temp_time = pd.DataFrame(columns = ['dates'])
df_temp_sub = pd.DataFrame(columns = ['subs'])

#turning time and subID from sorted psytoolkit file into lists so we can manipulate
time_list = df_psytoolkit_sorted[start_date_column].tolist()
sub_list = df_psytoolkit_sorted[sub_id_column].tolist()
flanker_file_list = df_psytoolkit_sorted[flanker_file_column].tolist()
questionnaire_file_list = df_psytoolkit_sorted[questionnaire_file_column].tolist()
        
#turning dates from date plus time to only date -- so we can match with daily task start dates from redcap file    
for y in time_list:
    current_file = y #selecting current file based on looping y
    current_file_date = current_file[0:10] #selecting first 6 characters, which are subID
    df_temp_time = df_temp_time.append({'dates' : str(current_file_date)}, ignore_index = True) #appending temp list 
    
#turning temp time list into list    
temp_time_list = df_temp_time['dates'].tolist()

print('done!')

done!


In [18]:
#MATCHING PARTICIPANTS TO DAILY TASK DATES; whole list
      
#reading in the subs file
col_list = ["subIDs"]
df = pd.read_csv(r'' + sub_dir + subs_filename, usecols=col_list) #read in file
df["subIDs"] = df["subIDs"].apply(str)

#turning excel column into list so we can manipulate the data
df_updating_subs = df["subIDs"].tolist()    
    
for x in df_updating_subs: #cycling through participant list one at a time
    loc_red = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc_red]==2: #checking if session has been completed
        
        #creating the sheets in the excel file where output will be written
        sheet1 = wb1.add_sheet(x)
        #putting headers in each excel sheet
        sheet1.write(0,0,'subID')
        sheet1.write(0,1,'ID type entered by participant')
        sheet1.write(0,2,'ID entered by participant')
        sheet1.write(0,3,'dates')
        sheet1.write(0,4,'day in daily tasks')
        sheet1.write(0,5,'number of files in psytoolkit for this participant on this day')
        sheet1.write(0,6,'missing?')

        #setting a counter to input data in the correct excel cell
        place_counter_complete=0
        sub_dailytasks_start = df_dailytasks_start[loc_red] #finding start date of daily tasks for x participant in redcap file
        
        for num_days in range(0,60): #60 daily tasks in total, starting on start date
            
            del df_temp_time #deleting/emptying the dataframe for each iteration
            del df_temp_sub #deleting/emptying the dataframe for each iteration
            df_temp_time = pd.DataFrame(columns = ['dates']) #setting the dataframe back up for current iteration
            df_temp_sub = pd.DataFrame(columns = ['subs']) #setting the dataframe back up for current iteration
            
            #modifying date to cycle through 60 days of daily tasks (num_days variable)
            date = datetime.strptime(sub_dailytasks_start, "%Y-%m-%d")
            modified_date = date + timedelta(days=num_days)
            datetime.strftime(modified_date, "%Y-%m-%d")
            current_day = datetime.strftime(modified_date, "%Y-%m-%d")
            
            date_indices = [i for i, dates in enumerate(temp_time_list) if dates == current_day] #creating list to index all cells that match the current date 
            
            #checking if date_indices is empty; if so, end
            if not date_indices:
                place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                sheet1.write(place_counter_complete, 1, 'missing') 
                sheet1.write(place_counter_complete, 2, 'missing') #write the ID entered as missing
                sheet1.write(place_counter_complete, 3, current_day) #write the date
                sheet1.write(place_counter_complete, 4, ['day ' + str(num_days+1)]) #write the daily task day
                sheet1.write(place_counter_complete, 5, 0) #writing number of files found for this participant on this day
                sheet1.write(place_counter_complete, 6, 'MISSING') #writing missing column
                continue
                
            #creating a sub list that only includes the subids listed in the date index -- so matching indices of all start dates and all subs on those days
            for z in date_indices:  
                current_sub_index = sub_list[z] 
                df_temp_sub = df_temp_sub.append({'subs' : str(current_sub_index)}, ignore_index = True) #appending temp list 
                temp_sub_list = df_temp_sub['subs'].tolist()
                
            df_text_file = pd.DataFrame({'subs': sub_list, 'dates': temp_time_list, 'flanker files': flanker_file_list, 'questionnaire files': questionnaire_file_list })
            
            #checking if any of the 3 IDs for this particular participant is in the temp list of subIDs on that start date
            if df_red_id[loc_red] in temp_sub_list:
                place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                occurrences = temp_sub_list.count(df_red_id[loc_red])
                sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                sheet1.write(place_counter_complete, 1, 'AOL ID') #write the ID type
                sheet1.write(place_counter_complete, 2, df_red_id[loc_red]) #write the ID entered
                sheet1.write(place_counter_complete, 3, current_day) #write the date
                sheet1.write(place_counter_complete, 4, ['day ' + str(num_days+1)]) #write the daily task day
                sheet1.write(place_counter_complete, 5, occurrences) #writing number of files found for this participant on this day
                continue 
            if df_red_id[loc_red] not in temp_sub_list:
                if df_phit_id[loc_red] in temp_sub_list:
                    place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                    occurrences = temp_sub_list.count(df_phit_id[loc_red])
                    sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                    sheet1.write(place_counter_complete, 1, 'PHIT ID') #write the ID type
                    sheet1.write(place_counter_complete, 2, df_phit_id[loc_red]) #write the ID entered
                    sheet1.write(place_counter_complete, 3, current_day) #write the date
                    sheet1.write(place_counter_complete, 4, ['day ' + str(num_days+1)]) #write the daily task day
                    sheet1.write(place_counter_complete, 5, occurrences) #writing number of files found for this participant on this day
                    continue
                if df_phit_id[loc_red] not in temp_sub_list:
                    if df_task_id[loc_red] in temp_sub_list:
                        place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                        occurrences = temp_sub_list.count(df_task_id[loc_red])
                        sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                        sheet1.write(place_counter_complete, 1, 'TASK ID') #write the ID type
                        sheet1.write(place_counter_complete, 2, df_task_id[loc_red]) #write the ID entered
                        sheet1.write(place_counter_complete, 3, current_day) #write the date
                        sheet1.write(place_counter_complete, 4, ['day ' + str(num_days+1)]) #write the daily task day
                        sheet1.write(place_counter_complete, 5, occurrences) #writing number of files found for this participant on this day
                        continue
                    if df_task_id[loc_red] not in temp_sub_list:
                        place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                        sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                        sheet1.write(place_counter_complete, 1, 'missing') #write the ID type as missing
                        sheet1.write(place_counter_complete, 2, 'missing') #write the ID entered as missing
                        sheet1.write(place_counter_complete, 3, current_day) #write the date
                        sheet1.write(place_counter_complete, 4, ['day ' + str(num_days+1)]) #write the daily task day
                        sheet1.write(place_counter_complete, 5, 0) #writing number of files found for this participant on this day
                        sheet1.write(place_counter_complete, 6, 'MISSING') #writing missing column   
    
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
print('done!')

done!


In [19]:
# RECENT DAYS CHECK

#reading in the subs file
col_list = ["subIDs"]
df = pd.read_csv(r'' + sub_dir + subs_filename_recent_days, usecols=col_list) #read in file
df["subIDs"] = df["subIDs"].apply(str)

#turning excel column into list so we can manipulate the data
df_updating_subs = df["subIDs"].tolist()

#opening a new workbook
wb2 = Workbook()

#creating empty temp dataframes to use later
df_temp_time = pd.DataFrame(columns = ['dates'])
df_temp_sub = pd.DataFrame(columns = ['subs'])

#turning time and subID from sorted psytoolkit file into lists so we can manipulate
time_list = df_psytoolkit_sorted[start_date_column].tolist()
sub_list = df_psytoolkit_sorted[sub_id_column].tolist()
flanker_file_list = df_psytoolkit_sorted[flanker_file_column].tolist()
questionnaire_file_list = df_psytoolkit_sorted[questionnaire_file_column].tolist()
        
#turning dates from date plus time to only date -- so we can match with daily task start dates from redcap file    
for y in time_list:
    current_file = y #selecting current file based on looping y
    current_file_date = current_file[0:10] #selecting first 6 characters, which are subID
    df_temp_time = df_temp_time.append({'dates' : str(current_file_date)}, ignore_index = True) #appending temp list 
    
#turning temp time list into list    
temp_time_list = df_temp_time['dates'].tolist()

#getting the current date
date_today =str(datetime.now())
date_today = date_today[0:10]
            
for x in df_updating_subs:
    loc_red = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc_red]==2: #checking if session has been completed
        
        #creating the sheets in the excel file where output will be written
        sheet1 = wb2.add_sheet(x)
        #putting headers in each excel sheet
        sheet1.write(0,0,'subID')
        sheet1.write(0,1,'ID type entered by participant')
        sheet1.write(0,2,'ID entered by participant')
        sheet1.write(0,3,'dates')
        sheet1.write(0,4,'day in daily tasks')
        sheet1.write(0,5,'number of files in psytoolkit for this participant on this day')
        sheet1.write(0,6,'missing?')

        #setting a counter to input data in the correct excel cell
        place_counter_complete=0
        
        sub_dailytasks_start = df_dailytasks_start[loc_red] #finding start date of daily tasks for x participant in redcap file
        
        for num_days in range(0,num_days_to_check): # past however many daily tasks in total, starting on today's today. set num_days_to_check in setup cell
            
            del df_temp_time #deleting/emptying the dataframe for each iteration
            del df_temp_sub #deleting/emptying the dataframe for each iteration
            df_temp_time = pd.DataFrame(columns = ['dates']) #setting the dataframe back up for this iteration
            df_temp_sub = pd.DataFrame(columns = ['subs']) #setting the dataframe back up for this iteration
            
            #modifying date to get past days (set by num_days_to_check and num_days)
            modified_date = datetime.now() - timedelta(days=num_days)
            datetime.strftime(modified_date, "%Y-%m-%d")
            current_day = datetime.strftime(modified_date, "%Y-%m-%d")
            
            #modifying date to see what day of daily tasks they are on (based on start day of daily tasks as entered in redcap)
            start_date = datetime.strptime(sub_dailytasks_start, "%Y-%m-%d")
            current_date = datetime.strptime(current_day, "%Y-%m-%d")

            days_passed = current_date - start_date
            #print(days_passed.days)

            #loc_time_list_startdates = temp_time_list.index(sub_dailytasks_start) #finding location of 
            date_indices = [i for i, dates in enumerate(temp_time_list) if dates == current_day] #creating list to index all cells that match the current date 
        
            #creating a sub list that only includes the subids listed in the date index -- so matching indices of all start dates and all subs on those days
            for z in date_indices:   
                current_sub_index = sub_list[z] 
                df_temp_sub = df_temp_sub.append({'subs' : str(current_sub_index)}, ignore_index = True) #appending temp list 
                temp_sub_list = df_temp_sub['subs'].tolist()
            
            #checking if any of the 3 IDs for this particular participant is in the temp list of subIDs on that start date
            if df_red_id[loc_red] in temp_sub_list:
                place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                occurrences = temp_sub_list.count(df_red_id[loc_red])
                sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                sheet1.write(place_counter_complete, 1, 'AOL ID') #write the date
                sheet1.write(place_counter_complete, 2, df_red_id[loc_red]) #write the ID entered
                sheet1.write(place_counter_complete, 3, current_day) #write the date
                sheet1.write(place_counter_complete, 4, ['day ' + str(days_passed.days)]) #write the daily task day
                sheet1.write(place_counter_complete, 5, occurrences) #writing number of files found for this participant on this day
                sheet1.write(place_counter_complete, 6, 'complete') #writing missing column
                continue 
            if df_red_id[loc_red] not in temp_sub_list:
                if df_phit_id[loc_red] in temp_sub_list:
                    place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                    sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                    sheet1.write(place_counter_complete, 1, 'PHIT ID') #write the date
                    sheet1.write(place_counter_complete, 2, df_phit_id[loc_red]) #write the ID entered
                    sheet1.write(place_counter_complete, 3, current_day) #write the date
                    sheet1.write(place_counter_complete, 4, ['day ' + str(days_passed.days)]) #write the daily task day
                    sheet1.write(place_counter_complete, 5, occurrences) #writing number of files found for this participant on this day
                    sheet1.write(place_counter_complete, 6, 'complete') #writing missing column
                    continue
                if df_phit_id[loc_red] not in temp_sub_list:
                    if df_task_id[loc_red] in temp_sub_list:
                        place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                        sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                        sheet1.write(place_counter_complete, 1, 'TASK ID') #write the date
                        sheet1.write(place_counter_complete, 2, df_task_id[loc_red]) #write the ID entered
                        sheet1.write(place_counter_complete, 3, current_day) #write the date
                        sheet1.write(place_counter_complete, 4, ['day ' + str(days_passed.days)]) #write the daily task day
                        sheet1.write(place_counter_complete, 5, occurrences) #writing number of files found for this participant on this day
                        sheet1.write(place_counter_complete, 6, 'complete') #writing missing column
                        continue
                    if df_task_id[loc_red] not in temp_sub_list:
                        place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                        sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
                        sheet1.write(place_counter_complete, 1, 'missing') 
                        sheet1.write(place_counter_complete, 2, 'missing') #write the ID entered as missing
                        sheet1.write(place_counter_complete, 3, current_day) #write the date
                        sheet1.write(place_counter_complete, 4, ['day ' + str(days_passed.days)]) #write the daily task day
                        sheet1.write(place_counter_complete, 5, 0) #writing number of files found for this participant on this day
                        sheet1.write(place_counter_complete, 6, 'MISSING') #writing missing column
        
#saving the workborks .xlsx files        
wb2.save(save_dir + save_name_recent_days)
print('done!')

done!
